<a href="https://colab.research.google.com/github/navin2065/Text-Summarization/blob/main/Another_copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q spacy nltk scikit-learn networkx gradio pdfminer.six

# Download necessary models
import nltk
nltk.download("punkt")

import spacy
spacy.cli.download("en_core_web_sm")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import numpy as np
import networkx as nx
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from pdfminer.high_level import extract_text

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess and tokenize sentences
def preprocess_sentences(text):
    sentences = sent_tokenize(text)
    cleaned = [sent.strip().replace("\n", " ") for sent in sentences if len(sent.strip()) > 0]
    return cleaned

# Function to compute sentence embeddings using spaCy
def sentence_embeddings(sentences):
    return [nlp(sentence).vector for sentence in sentences]

# Build similarity matrix between sentence embeddings
def build_similarity_matrix(embeddings):
    sim_matrix = np.zeros((len(embeddings), len(embeddings)))
    for i in range(len(embeddings)):
        for j in range(len(embeddings)):
            if i != j:
                sim_matrix[i][j] = cosine_similarity(
                    embeddings[i].reshape(1, -1),
                    embeddings[j].reshape(1, -1)
                )[0, 0]
    return sim_matrix

# Summarizer using TextRank
def summarize_text(text, num_sentences=None):
    sentences = preprocess_sentences(text)
    if len(sentences) == 0:
        return "No valid sentences found."

    embeddings = sentence_embeddings(sentences)
    sim_matrix = build_similarity_matrix(embeddings)
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)

    if num_sentences is None or num_sentences > len(sentences):
        num_sentences = len(sentences)

    selected = sorted(ranked_sentences[:num_sentences], key=lambda x: x[2])
    return " ".join([s[1] for s in selected])


In [ ]:
# Function to handle file input (txt and pdf)
def extract_text_from_file(file):
    if file.name.endswith(".txt"):
        # Read .txt file
        text = file.read().decode("utf-8")
    elif file.name.endswith(".pdf"):
        # Extract text from PDF
        text = extract_text(file)
    else:
        text = None
    return text


In [ ]:
import gradio as gr

# Gradio interface function
def gradio_summarizer(file, num_sentences):
    # Extract text from the uploaded file
    text = extract_text_from_file(file)

    if not text:
        return "Invalid file format. Please upload a .txt or .pdf file."

    # Check if the file is empty or contains invalid content
    if len(text.strip()) == 0:
        return "The uploaded file is empty. Please upload a valid file."

    # Summarize the extracted text
    summary = summarize_text(text, num_sentences)
    return summary

# Create Gradio interface
iface = gr.Interface(
    fn=gradio_summarizer,
    inputs=[
        gr.File(file_count="single", type="filepath", label="Upload a Text or PDF File"),  # File Upload: Changed type to "filepath"
        gr.Slider(minimum=1, maximum=100, step=1, value=5, label="Number of Sentences to Summarize")  # Number of sentences slider: Changed 'default' to 'value'
    ],
    outputs=gr.Textbox(label="Summary", lines=15),  # Summary output with more lines for longer text
    live=True  # Show summary in real-time
)

# Launch the interface
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e87a29ee6486bfad25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -q transformers gradio pymupdf

from transformers import pipeline
import gradio as gr
import fitz  # PyMuPDF

# Load summarization pipeline
try:
    summarizer = pipeline("summarization")
except Exception as e:
    print("Error loading summarization model:", e)

def extract_text_from_pdf(filepath):
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        return None, f"Error extracting PDF text: {e}"

def summarize_text(file):
    if file is None:
        return "Please upload a file (.txt or .pdf)."

    filename = file.name.lower()

    # Handle txt file
    if filename.endswith(".txt"):
        try:
            with open(file.name, "r", encoding="utf-8") as f:
                text = f.read()
        except UnicodeDecodeError:
            return "Error decoding .txt file. Please upload a UTF-8 encoded text file."
        except Exception as e:
            return f"Error reading file: {e}"

    # Handle pdf file
    elif filename.endswith(".pdf"):
        text, error = extract_text_from_pdf(file.name)
        if text is None:
            return error
    else:
        return "Unsupported file format. Please upload .txt or .pdf files only."

    if len(text.strip()) == 0:
        return "Uploaded file is empty or no extractable text found."

    max_len = 3000
    if len(text) > max_len:
        text = text[:max_len] + "..."

    try:
        summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Error during summarization: {e}"

iface = gr.Interface(
    fn=summarize_text,
    inputs=gr.File(file_types=[".txt", ".pdf"], label="Upload a text or PDF file"),
    outputs="text",
    title="Text & PDF Summarization Tool",
    description="Upload a .txt or .pdf file and get an NLP-generated summary."
)

iface.launch(share=True)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e50015a34488fb32ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -q transformers gradio pymupdf

from transformers import pipeline
import gradio as gr
import fitz

# Use a smaller, faster summarization model
try:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
except Exception as e:
    print("Error loading summarization model:", e)

def extract_text_from_pdf(filepath):
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        return None, f"Error extracting PDF text: {e}"

def chunk_text(text, max_words=600):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)
    return chunks

def summarize_text(file, num_sentences):
    if file is None:
        return "Please upload a file (.txt or .pdf)."

    filename = file.name.lower()

    if filename.endswith(".txt"):
        try:
            with open(file.name, "r", encoding="utf-8") as f:
                text = f.read()
        except Exception as e:
            return f"Error reading file: {e}"
    elif filename.endswith(".pdf"):
        text, error = extract_text_from_pdf(file.name)
        if text is None:
            return error
    else:
        return "Unsupported file format. Please upload .txt or .pdf files only."

    if not text.strip():
        return "Uploaded file is empty or no extractable text found."

    # Limit text length to first 5000 characters for speed
    text = text[:5000]

    chunks = chunk_text(text, max_words=600)

    # Limit number of chunks to 3 for speed
    chunks = chunks[:3]

    max_length_tokens = max(30, min(num_sentences * 20, 512))
    min_length_tokens = max(5, max_length_tokens // 4)

    summaries = []
    for chunk in chunks:
        try:
            summary = summarizer(
                chunk,
                max_length=max_length_tokens,
                min_length=min_length_tokens,
                do_sample=False
            )
            summaries.append(summary[0]['summary_text'])
        except Exception as e:
            summaries.append(f"[Error summarizing chunk: {e}]")

    # Join summaries without re-summarizing for speed
    return "\n\n".join(summaries)

iface = gr.Interface(
    fn=summarize_text,
    inputs=[
        gr.File(file_types=[".txt", ".pdf"], label="Upload a text or PDF file"),
        gr.Slider(1, 500, step=1, value=20, label="Number of sentences in summary (max 100 for speed)")
    ],
    outputs="text",
    title="Fast Text & PDF Summarization Tool",
    description="Upload a .txt or .pdf file and choose number of sentences (up to 100) for a fast summary."
)

iface.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1c2e1f1228411b3e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -q transformers gradio pymupdf python-docx

from transformers import pipeline
import gradio as gr
import fitz  # PyMuPDF
import docx

# Load summarization pipeline
summarizer = pipeline("summarization")

def extract_text_from_pdf(filepath):
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        return None, f"Error extracting PDF text: {e}"

def extract_text_from_docx(filepath):
    try:
        doc = docx.Document(filepath)
        fullText = []
        for para in doc.paragraphs:
            fullText.append(para.text)
        return "\n".join(fullText)
    except Exception as e:
        return None, f"Error extracting DOCX text: {e}"

def summarize_text(file, num_sentences):
    if file is None:
        return "Please upload a file (.txt, .pdf or .docx)."

    filename = file.name.lower()

    if filename.endswith(".txt"):
        try:
            with open(file.name, "r", encoding="utf-8") as f:
                text = f.read()
        except UnicodeDecodeError:
            return "Error decoding .txt file. Please upload a UTF-8 encoded text file."
        except Exception as e:
            return f"Error reading file: {e}"

    elif filename.endswith(".pdf"):
        text, error = extract_text_from_pdf(file.name)
        if text is None:
            return error

    elif filename.endswith(".docx"):
        text, error = extract_text_from_docx(file.name)
        if text is None:
            return error
    else:
        return "Unsupported file format. Please upload .txt, .pdf or .docx files only."

    if len(text.strip()) == 0:
        return "Uploaded file is empty or no extractable text found."

    max_chars = 3000
    if len(text) > max_chars:
        text = text[:max_chars] + "..."

    max_length = num_sentences * 15
    min_length = max(5, num_sentences * 5)

    try:
        summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Error during summarization: {e}"

def save_summary(summary_text):
    if isinstance(summary_text, str) and len(summary_text.strip()) > 0:
        filename = "summary.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(summary_text)
        return filename
    return None

iface = gr.Interface(
    fn=summarize_text,
    inputs=[
        gr.File(file_types=[".txt", ".pdf", ".docx"], label="Upload a text, PDF, or Word (.docx) file"),
        gr.Slider(minimum=1, maximum=10, step=1, value=3, label="Number of sentences in summary"),
    ],
    outputs=gr.Textbox(label="Summary"),
    title="Multiformat Text Summarization Tool",
    description="Upload .txt, .pdf or .docx files and get an NLP-generated summary. Adjust summary length.",
    allow_flagging="never",
)

# Add a download button for the summary output
download_btn = gr.File(label="Download Summary", interactive=False)

def summarize_and_return_file(file, num_sentences):
    summary = summarize_text(file, num_sentences)
    # Save summary to file for download
    if isinstance(summary, str) and not summary.startswith("Error") and not summary.startswith("Please"):
        filename = "summary.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(summary)
        return summary, filename
    else:
        return summary, None

iface2 = gr.Interface(
    fn=summarize_and_return_file,
    inputs=[
        gr.File(file_types=[".txt", ".pdf", ".docx"], label="Upload a file"),
        gr.Slider(minimum=1, maximum=10, step=1, value=3, label="Number of sentences"),
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.File(label="Download Summary")
    ],
    title="Summarization with Download",
    description="Upload a .txt, .pdf or .docx file and get a summary with an option to download.",
)

iface2.launch(share=True)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4979563ddc812ac339.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
